In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Create manifest

In [21]:
import re, json, pandas as pd, numpy as np, tqdm
from pathlib import Path

# ------------------------------------------------------------
# 0 .  configurable paths & regex dictionary
# ------------------------------------------------------------
ROOT = Path(r"/content/drive/MyDrive/Trustworthy_AI_Final_Project/CBM_w_AneRBC/AneRBC_full/AneRBC/AneRBC_dataset/AneRBC-I")

regex_dict_filtered = {
    # RBC size / colour
    "microcytic"      : r"\bmicrocytic\b",
    "normocytic"      : r"\bnormocytic\b",
    "normochromic"    : r"\bnormochromic\b",
    "hypochromic"     : r"\bhypochromic\b",

    # RBC shape & inclusions
    "elliptocytes"    : r"\belliptocytes?\b",
    "target_cells"    : r"\btarget\s+cells?\b",
    "tear_drop_cells" : r"\btear\s?drop\s+cells?\b",
    "anisocytosis"    : r"\banisocytosis\b",

    # RBC regeneration / stacking
    "polychromasia"   : r"\bpolychromasia\b",

    # WBC abnormalities
    "neutrophilia"    : r"\bneutrophilia\b",
    "monocytosis"     : r"\bmonocytosis\b",
    "reactive_lymphocytes": r"\breactive\s+lymphocytes?\b",

    # Platelet findings
    "platelets_increased": r"\bplatelets?\s+(increased|raised|elevated)\b",
    "platelets_decreased": r"\bplatelets?\s+(decreased|reduced|low)\b",

    # Disease / context
    "thalassemia"     : r"\bthalassemia\b",
}

CBC_CANON = {
    # canonical : list of regex patterns covering spelling variants
    "HGB": [r"\bHGB\b", r"\bH\.?emoglobin\b", r"\bHemoglobin\b"],
    "HCT": [r"\bHCT\b", r"\bH\.?ematocrit\b", r"\bHematocrit\b"],
    "RBC": [r"\bRBC\b", r"\bRed\s+Blood\s+Cells?\b"],
    "MCV": [r"\bMCV\b", r"\bMean\s+Corpuscular\s+Volume\b"],
    "MCH": [r"\bMCH\b", r"\bMean\s+Corpuscular\s+Hemoglobin\b"],
}

CBC_RE = {k: re.compile('|'.join(pat), re.I) for k, pat in CBC_CANON.items()}
VALUE_RE = re.compile(r"([0-9]*\.?[0-9]+)")

# ------------------------------------------------------------
# 1 .  helper functions
# ------------------------------------------------------------
def parse_keywords(text: str, regex_dict: dict) -> dict:
    """Return {keyword:0/1} according to presence in report."""
    return {k: int(bool(re.search(rx, text, flags=re.I)))
            for k, rx in regex_dict.items()}

def parse_cbc(text: str) -> dict:
    out = {k: None for k in CBC_CANON}
    for line in text.splitlines():
        for key, rx in CBC_RE.items():
            if rx.search(line):
                m = VALUE_RE.search(line)
                if m:
                    out[key] = float(m.group(1))
    # fallback: set missing values to 0.0 or NaN
    return {k: (v if v is not None else 0.0) for k, v in out.items()}


# def parse_cbc(text: str) -> dict:
#     """OLD: Return {CBC_name: float_value} from one CBC report file."""
#     out = {}
#     for line in text.splitlines():
#         m = CBC_RE.match(line)
#         if m:
#             name, val = m.groups()
#             out[name] = float(val)
#     return out

# ------------------------------------------------------------
# 2 .  walk dataset & build manifest rows
# ------------------------------------------------------------
rows = []
for split in ["Healthy_individuals", "Anemic_individuals"]:
    base = ROOT / split
    for cbc_path in tqdm.tqdm((base / "CBC_reports").glob("*.txt")):
        stem        = cbc_path.stem                      # e.g. '001_a'

        # Run the bad data exploration code first. This will skip all the files with bad data.
        if stem in stem_hist:
          continue

        morph_path  = base / "Morphology_reports" / f"{stem}.txt"
        img_path    = base / "Original_images"   / f"{stem}.png"

        kw_flags    = parse_keywords(morph_path.read_text(), regex_dict_filtered)
        cbc_vals    = parse_cbc(cbc_path.read_text())

        concept_dict = {**kw_flags, **cbc_vals}

        rows.append({
            "image"    : str(img_path),
            "concepts" : json.dumps(concept_dict),
            "label"    : int(stem.endswith("_a"))   # 1=anemic, 0=healthy
        })

# ------------------------------------------------------------
# 3 .  save CSV manifest
# ------------------------------------------------------------
pd.DataFrame(rows).to_csv("anerbc_manifest.csv", index=False)
print("saved", len(rows), "rows → anerbc_manifest.csv")

500it [03:19,  2.50it/s]
500it [03:32,  2.35it/s]

saved 876 rows → anerbc_manifest_only_hgb.csv


# Exploring dataset: Finding all unique CBC tests and its variations in spelling

In [ ]:
import re, json, pandas as pd, numpy as np, tqdm
from pathlib import Path

ROOT = Path(r"/content/drive/MyDrive/Trustworthy_AI_Final_Project/CBM_w_AneRBC/AneRBC_full/AneRBC/AneRBC_dataset/AneRBC-I")

count = 0
COUNT_BRK = 1
vocab_count_dict = {}
for split in ["Healthy_individuals", "Anemic_individuals"]:
    base = ROOT / split
    for cbc_path in tqdm.tqdm((base / "CBC_reports").glob("*.txt")):
        stem        = cbc_path.stem                      # e.g. '001_a'
        morph_path  = base / "Morphology_reports" / f"{stem}.txt"
        img_path    = base / "Original_images"   / f"{stem}.png"

        text = cbc_path.read_text().splitlines()
        # skip first value
        for line in text[1:]:
          line_test = line.split(',')[0]
          if line_test not in vocab_count_dict:
            print(line_test)
            print(f"{stem}.png")
            vocab_count_dict[line_test] = 1
          else:
            vocab_count_dict[line_test] += 1

    #     count += 1
    #     if count == COUNT_BRK:
    #       break
    # if count == COUNT_BRK:
    #   break

# print(vocab_count_dict)




71it [00:00, 361.10it/s]

WBC
013_h.png
RBC
013_h.png
HGB
013_h.png
HCT
013_h.png
MCV
013_h.png
MCH
013_h.png
MCHC
013_h.png
RDW---CV
013_h.png
PLT
013_h.png
MPV
013_h.png
Neut
013_h.png
LYMP
013_h.png
MONO
013_h.png
EOS
013_h.png
BASO
013_h.png
#NEUT
013_h.png
#LYMP
013_h.png
#MONO
013_h.png
#EOS
013_h.png
#BASO
013_h.png
#LYMP6.89 x10.e 3/μl
007_h.png
#EOS0.13 x10.e 3/μl
005_h.png
%Neut
076_h.png
%LYMP
076_h.png
%MONO
076_h.png
%EOS
076_h.png
%BASO
076_h.png
%RDW---CV
077_h.png


149it [00:00, 373.98it/s]

Neutrophils
074_h.png
Lymphocytes
074_h.png
Monocytes
074_h.png
Eosinophils
074_h.png
%RDW-CV
136_h.png
%EOSx10.e 3/μl
157_h.png


224it [00:00, 360.35it/s]

...

191it [00:00, 384.28it/s]

#RASO
109_a.png
RDW-CV
134_a.png
ΜΟΝΟ
134_a.png
#ΜΟΝΟ
134_a.png
$MONO
175_a.png
$BASO
175_a.png
$LYMP
176_a.png


268it [00:00, 370.29it/s]

&BASO
237_a.png
SEOS
229_a.png
MCHO
216_a.png
&RDW---CV
216_a.png
$Neut
238_a.png
#Neut
227_a.png


500it [00:01, 372.94it/s]


# Exploring dataset: Finding files with bad data

In [16]:
# ------------------------------------------------------------
# walk dataset
# ------------------------------------------------------------
import re, json, pandas as pd, numpy as np
from pathlib import Path

ROOT = Path(r"/content/drive/MyDrive/Trustworthy_AI_Final_Project/CBM_w_AneRBC/AneRBC_full/AneRBC/AneRBC_dataset/AneRBC-I")

CBC_CANON = {
    # canonical : list of regex patterns covering spelling variants
    "HGB": [r"\bHGB\b", r"\bH\.?emoglobin\b", r"\bHemoglobin\b"],
    "HCT": [r"\bHCT\b", r"\bH\.?ematocrit\b", r"\bHematocrit\b"],
    "RBC": [r"\bRBC\b", r"\bRed\s+Blood\s+Cells?\b"],
    "MCV": [r"\bMCV\b", r"\bMean\s+Corpuscular\s+Volume\b"],
    "MCH": [r"\bMCH\b", r"\bMean\s+Corpuscular\s+Hemoglobin\b"],
}

CBC_RE = {k: re.compile('|'.join(pat), re.I) for k, pat in CBC_CANON.items()}
VALUE_RE = re.compile(r"([0-9]*\.?[0-9]+)")

PLR = {                       # hard lower / upper limits
    "RBC":  (1.5,  8),
    "HGB":  (5.0,  20.0),
    "HCT":  (15.0, 65.0),
    "MCV":  (50.0, 130.0),
    "MCH":  (15.0, 40.0)
    }

rows = []
stem_hist = []
for split in ["Healthy_individuals", "Anemic_individuals"]:
    base = ROOT / split
    for cbc_path in (base / "CBC_reports").glob("*.txt"):
        stem        = cbc_path.stem                      # e.g. '001_a'
        morph_path  = base / "Morphology_reports" / f"{stem}.txt"
        img_path    = base / "Original_images"   / f"{stem}.png"

        cbc_vals    = parse_cbc(cbc_path.read_text())

        if cbc_vals["HGB"] < PLR["HGB"][0]:
          print(f"Low value {cbc_vals['HGB']} at {stem}.png")
          if stem not in stem_hist:
            stem_hist.append(stem)
        elif cbc_vals["HGB"] > PLR["HGB"][1]:
          print(f"High value {cbc_vals['HGB']} at {stem}.png")
          if stem not in stem_hist:
            stem_hist.append(stem)

        if cbc_vals["RBC"] < PLR["RBC"][0]:
          print(f"Low value {cbc_vals['RBC']} at {stem}.png")
          if stem not in stem_hist:
            stem_hist.append(stem)
        elif cbc_vals["RBC"] > PLR["RBC"][1]:
          print(f"High value {cbc_vals['RBC']} at {stem}.png")
          if stem not in stem_hist:
            stem_hist.append(stem)

        if cbc_vals["HCT"] < PLR["HCT"][0]:
          print(f"Low value {cbc_vals['HCT']} at {stem}.png")
          if stem not in stem_hist:
            stem_hist.append(stem)
        elif cbc_vals["HCT"] > PLR["HCT"][1]:
          print(f"High value {cbc_vals['HCT']} at {stem}.png")
          if stem not in stem_hist:
            stem_hist.append(stem)

        if cbc_vals["MCV"] < PLR["MCV"][0]:
          print(f"Low value {cbc_vals['MCV']} at {stem}.png")
          if stem not in stem_hist:
            stem_hist.append(stem)
        elif cbc_vals["MCV"] > PLR["MCV"][1]:
          print(f"High value {cbc_vals['MCV']} at {stem}.png")
          if stem not in stem_hist:
            stem_hist.append(stem)

        if cbc_vals["MCH"] < PLR["MCH"][0]:
          print(f"Low value {cbc_vals['MCH']} at {stem}.png")
          if stem not in stem_hist:
            stem_hist.append(stem)
        elif cbc_vals["MCH"] > PLR["MCH"][1]:
          print(f"High value {cbc_vals['MCH']} at {stem}.png")
          if stem not in stem_hist:
            stem_hist.append(stem)

print(f"{len(stem_hist)} files affected")

High value 8.29 at 013_h.png
Low value 4.8 at 025_h.png
Low value 4.3 at 059_h.png
Low value 1.27 at 059_h.png
Low value 12.3 at 059_h.png
High value 88.0 at 076_h.png
High value 13.09 at 084_h.png
High value 8.6 at 083_h.png
Low value 7.1 at 083_h.png
Low value 13.8 at 083_h.png
Low value 13.6 at 083_h.png
Low value 1.48 at 067_h.png
Low value 5.5 at 067_h.png
High value 3646.0 at 067_h.png
High value 10.53 at 072_h.png
Low value 46.8 at 086_h.png
Low value 12.3 at 086_h.png
High value 10.53 at 073_h.png
Low value 9.6 at 082_h.png
Low value 34.2 at 082_h.png
High value 53.7 at 082_h.png
Low value 1.48 at 066_h.png
Low value 5.5 at 066_h.png
High value 3646.0 at 066_h.png
Low value 4.5 at 106_h.png
Low value 12.6 at 106_h.png
Low value 3.84 at 092_h.png
Low value 4.01 at 093_h.png
High value 13.28 at 093_h.png
Low value 7.2 at 093_h.png
Low value 25.9 at 093_h.png
High value 64.6 at 093_h.png
Low value 4.7 at 097_h.png
Low value 14.5 at 097_h.png
Low value 14.8 at 107_h.png
High value 

In [18]:
count_h = 0
count_a = 0
for item in stem_hist:
  if "h" in item:
    count_h += 1
  else:
    count_a += 1
print(count_h)
print(count_a)

59
65


In [25]:
import pandas as pd
import json
import torch

# Load your manifest CSV
csv_path = "/content/drive/MyDrive/Trustworthy_AI_Final_Project/CBM_w_AneRBC/anerbc_manifest_filtered_out_bad_data.csv"
df = pd.read_csv(csv_path)

# Get all concept keys
all_keys = set()
for js in df.concepts:
    all_keys.update(json.loads(js).keys())

# Sort concept keys to ensure consistent order
concept_names = sorted(list(all_keys))

# Concepts to exclude (set prevalence to 0)
exclude_concepts = ["HGB", "RBC", "HCT", "MCV", "MCH"]

# Calculate prevalence for each concept (except those in exclude_concepts)
prevalence_dict = {}
for concept in concept_names:
    if concept not in exclude_concepts:
        total_count = len(df)
        positive_count = df['concepts'].apply(lambda x: json.loads(x).get(concept, 0) == 1).sum()
        prevalence_dict[concept] = positive_count / total_count

# Create prevalence tensor with prevalence of excluded concepts as 0
prevalence_list = [0.0] * len(exclude_concepts)  # Start with prevalence of excluded concepts as 0
for concept in concept_names:
    if concept not in exclude_concepts:
        prevalence_list.append(prevalence_dict[concept])

# Ensure prevalence list aligns with concept_names order
prevalence_list_ordered = []
for concept in concept_names:
    if concept in exclude_concepts:
        prevalence_list_ordered.append(0.0)  # Set prevalence to 0 for excluded concepts
    else:
        prevalence_list_ordered.append(prevalence_dict[concept])

device = 'cuda' if torch.cuda.is_available() else 'cpu'

prevalence = torch.tensor(prevalence_list_ordered, device=device)  # Use the ordered list

# Print concept names and their prevalence values
for i, concept in enumerate(concept_names):
    print(f"{concept:20s} {prevalence[i].item():.3f}")

HCT                  0.000
HGB                  0.000
MCH                  0.000
MCV                  0.000
RBC                  0.000
anisocytosis         0.300
elliptocytes         0.418
hypochromic          0.526
microcytic           0.486
monocytosis          0.055
neutrophilia         0.070
normochromic         0.471
normocytic           0.500
platelets_decreased  0.072
platelets_increased  0.140
polychromasia        0.234
reactive_lymphocytes 0.057
target_cells         0.205
tear_drop_cells      0.067
thalassemia          0.072


# Trying to find what CBC features exists in every report

In [12]:
import re, json, pandas as pd, numpy as np, tqdm
from pathlib import Path

ROOT = Path(r"/content/drive/MyDrive/Trustworthy_AI_Final_Project/CBM_w_AneRBC/AneRBC_full/AneRBC/AneRBC_dataset/AneRBC-I")

minimal_features = {
    1: {"keyword": ["HGB", "Hemoglobin"], "exists": True},
    2: {"keyword": ["HCT"],                 "exists": True},
    3: {"keyword": ["RBC"],                 "exists": True},
    4: {"keyword": ["MCV"],                 "exists": True},
    5: {"keyword": ["MCH"],                 "exists": True},
    6: {"keyword": ["MCHC"],                "exists": True},
    7: {"keyword": ["RDW---CV", "RDW-CV",
                    "%RDW---CV", "%RDW-CV",
                    "&RDW---CV", "$RDW---CV"],
        "exists": True}
}


# ------------------------------------------------------------
#  utility: normalise tokens so "MCHC", "mchc*", "MCHC " etc.
#  all collapse to "MCHC"
# ------------------------------------------------------------
def clean(tok: str) -> str:
    """
    Keep only A‑Z, 0‑9, convert to upper‑case.
    """
    return re.sub(r"[^A-Z0-9]", "", tok.upper())


# pre‑clean every keyword once
for feat in minimal_features.values():
    feat["clean_kw"] = {clean(k) for k in feat["keyword"]}

# ------------------------------------------------------------
#  walk every report
# ------------------------------------------------------------
for split in ["Healthy_individuals", "Anemic_individuals"]:
    reports_dir = ROOT / split / "CBC_reports"

    for cbc_path in tqdm.tqdm(reports_dir.glob("*.txt"), desc=f"Scanning {split}"):
        # collect all first‑column tokens for this one report
        tokens_in_report = set()

        for i, line in enumerate(cbc_path.read_text().splitlines()):
            if i == 0:                   # skip the leading header‑row if present
                continue                 # (<Test, Results/Units, Normal Range>)

            if not line.strip():         # blank line
                continue

            first_field = line.split(",")[0].strip()
            tokens_in_report.add(clean(first_field))

        # flip off any feature missing from *this* report
        for feat in minimal_features.values():
            # if feat["exists"]:
            if feat["clean_kw"].isdisjoint(tokens_in_report):
                if "RDWCV" in feat["clean_kw"]:
                    print(f"{cbc_path.stem}.txt")
                    print(tokens_in_report)
                feat["exists"] = False

# ------------------------------------------------------------
#  summary
# ------------------------------------------------------------
present_in_all = {
    fid: feat for fid, feat in minimal_features.items() if feat["exists"]
}

print("\nFeatures that appear in **every** CBC report:")
for fid, feat in present_in_all.items():
    print(f"{fid}: {feat['keyword'][0]}")


{1: {'keyword': ['HGB', 'Hemoglobin'], 'exists': True, 'clean_kw': {'HEMOGLOBIN', 'HGB'}}, 2: {'keyword': ['HCT'], 'exists': True, 'clean_kw': {'HCT'}}, 3: {'keyword': ['RBC'], 'exists': True, 'clean_kw': {'RBC'}}, 4: {'keyword': ['MCV'], 'exists': True, 'clean_kw': {'MCV'}}, 5: {'keyword': ['MCH'], 'exists': True, 'clean_kw': {'MCH'}}, 6: {'keyword': ['MCHC'], 'exists': True, 'clean_kw': {'MCHC'}}, 7: {'keyword': ['RDW---CV', 'RDW-CV', '%RDW---CV', '%RDW-CV', '&RDW---CV', '$RDW---CV'], 'exists': True, 'clean_kw': {'RDWCV'}}}


Scanning Healthy_individuals: 28it [00:00, 277.66it/s]

014_h.txt
{'MCH', 'WBC', 'MPV', 'HCT', 'PLT', 'MCHC', 'HGB', 'RBC', 'MCV'}
012_h.txt
{'MCH', 'WBC', 'MPV', 'HCT', 'PLT', 'MCHC', 'HGB', 'RBC', 'MCV'}
023_h.txt
{'MCH', 'WBC', 'MPV', 'HCT', 'PLT', 'MCHC', 'HGB', 'RBC', 'MCV'}
025_h.txt
{'EOS', 'BASO', 'MCH', 'MONO', 'WBC', 'HCT', 'PLT', 'MCHC', 'HGB', 'NEUT', 'LYMP', 'RBC', 'MCV'}
022_h.txt
{'MCH', 'WBC', 'HCT', 'PLT', 'MCHC', 'HGB', 'RBC', 'MCV'}
039_h.txt
{'MCH', 'WBC', 'MPV', 'HCT', 'PLT', 'MCHC', 'HGB', 'RBC', 'MCV'}
054_h.txt
{'MCH', 'WBC', 'MPV', 'HCT', 'PLT', 'MCHC', 'HGB', 'RBC', 'MCV'}
046_h.txt
{'MCH', 'WBC', 'MPV', 'HCT', 'PLT', 'MCHC', 'HGB', 'RBC', 'MCV'}


Scanning Healthy_individuals: 90it [00:00, 282.83it/s]

055_h.txt
{'MCH', 'WBC', 'MPV', 'HCT', 'PLT', 'MCHC', 'HGB', 'RBC', 'MCV'}
078_h.txt
{'MCH', 'WBC', 'HCT', 'PLT', 'MCHC', 'HGB', 'RBC', 'MCV'}
084_h.txt
{'MCH', 'WBC', 'HCT', 'PLT', 'MCHC', 'HGB', 'RBC', 'MCV'}
083_h.txt
{'EOS', 'BASO', 'MCH', 'MONO', 'WBC', 'MPV', 'HCT', 'PLT', 'MCHC', 'HGB', 'NEUT', 'LYMP', 'RBC', 'MCV'}
081_h.txt
{'EOS', 'BASO', 'MCH', 'MONO', 'WBC', 'HCT', 'PLT', 'MCHC', 'HGB', 'NEUT', 'LYMP', 'RBC', 'MCV'}
080_h.txt
{'EOS', 'BASO', 'MCH', 'MONO', 'WBC', 'MPV', 'HCT', 'PLT', 'MCHC', 'HGB', 'NEUT', 'LYMP', 'RBC', 'MCV'}
067_h.txt
{'EOS', 'BASO', 'MCH', 'MONO', 'WBC', 'MPV', 'HCT', 'PLT', 'MCHC', 'HGB', 'NEUT', 'LYMP', 'RBC', 'MCV'}
079_h.txt
{'EOS', 'BASO', 'MCH', 'MONO', 'WBC', 'HCT', 'PLT', 'MCHC', 'HGB', 'NEUT', 'LYMP', 'RBC', 'MCV'}
082_h.txt
{'EOS', 'BASO', 'MCH', 'MONO', 'WBC', 'HCT', 'PLT', 'MCHC', 'HGB', 'NEUT', 'LYMP', 'RBC', 'MCV'}
066_h.txt
{'EOS', 'BASO', 'MCH', 'MONO', 'WBC', 'MPV', 'HCT', 'PLT', 'MCHC', 'HGB', 'NEUT', 'LYMP', 'RBC', 'MCV'}
101_h.txt
{'

...

306_a.txt
{'HEMOGLOBIN', 'MCH', 'WBC', 'MPV', 'HCT', 'PLT', 'MCHC', 'RBC', 'MCV'}
299_a.txt
{'MCH', 'WBC', 'HCT', 'PLT', 'MCHC', 'HGB', 'RBC', 'MCV'}
332_a.txt
{'HEMOGLOBIN', 'MCH', 'WBC', 'HCT', 'PLT', 'MCHC', 'RBC', 'MCV'}
322_a.txt
{'HEMOGLOBIN', 'MCH', 'WBC', 'HCT', 'PLT', 'MCHC', 'RBC', 'MCV'}
329_a.txt
{'HEMOGLOBIN', 'MCH', 'WBC', 'MPV', 'HCT', 'PLT', 'MCHC', 'RBC', 'MCV'}
339_a.txt
{'HEMOGLOBIN', 'MCH', 'WBC', 'MPV', 'HCT', 'PLT', 'MCHC', 'RBC', 'MCV'}
342_a.txt
{'HEMOGLOBIN', 'MCH', 'WBC', 'HCT', 'PLT', 'MCHC', 'RBC', 'MCV'}
341_a.txt
{'HEMOGLOBIN', 'MCH', 'WBC', 'MPV', 'HCT', 'PLT', 'MCHC', 'RBC', 'MCV'}
343_a.txt
{'HEMOGLOBIN', 'MCH', 'WBC', 'MPV', 'HCT', 'PLT', 'MCHC', 'RBC', 'MCV'}
323_a.txt
{'HEMOGLOBIN', 'MCH', 'WBC', 'HCT', 'PLT', 'MCHC', 'RBC', 'MCV'}
331_a.txt
{'HEMOGLOBIN', 'MCH', 'WBC', 'HCT', 'PLT', 'MCHC', 'RBC', 'MCV'}
345_a.txt
{'HEMOGLOBIN', 'MCH', 'WBC', 'HCT', 'PLT', 'MCHC', 'RBC', 'MCV'}
340_a.txt
{'HEMOGLOBIN', 'MCH', 'WBC', 'HCT', 'PLT', 'MCHC', 'RBC', 'MCV

Scanning Anemic_individuals: 408it [00:01, 290.02it/s]

390_a.txt
{'MCH', 'WBC', 'MPV', 'HCT', 'PLT', 'MCHC', 'HGB', 'RBC', 'MCV'}
373_a.txt
{'MCH', 'WBC', 'HCT', 'PLT', 'MCHC', 'HGB', 'RBC', 'MCV'}
389_a.txt
{'MCH', 'WBC', 'MPV', 'HCT', 'PLT', 'MCHC', 'HGB', 'RBC', 'MCV'}
393_a.txt
{'MCH', 'WBC', 'MPV', 'HCT', 'PLT', 'MCHC', 'HGB', 'RBC', 'MCV'}
386_a.txt
{'MCH', 'WBC', 'MPV', 'HCT', 'PLT', 'MCHC', 'HGB', 'RBC', 'MCV'}
381_a.txt
{'MCH', 'WBC', 'HCT', 'PLT', 'MCHC', 'HGB', 'RBC', 'MCV'}
396_a.txt
{'EOS', 'BASO', 'MCH', 'MONO', 'WBC', 'HCT', 'PLT', 'MCHC', 'HGB', 'NEUT', 'LYMP', 'RBC', 'MCV'}
372_a.txt
{'MCH', 'WBC', 'MPV', 'HCT', 'PLT', 'MCHC', 'HGB', 'RBC', 'MCV'}
383_a.txt
{'MCH', 'WBC', 'HCT', 'PLT', 'MCHC', 'HGB', 'RBC', 'MCV'}
392_a.txt
{'MCH', 'WBC', 'HCT', 'PLT', 'MCHC', 'HGB', 'RBC', 'MCV'}
387_a.txt
{'MCH', 'WBC', 'MPV', 'HCT', 'PLT', 'MCHC', 'HGB', 'RBC', 'MCV'}
413_a.txt
{'MCH', 'WBC', 'MPV', 'HCT', 'PLT', 'MCHC', 'HGB', 'RBC', 'MCV'}
398_a.txt
{'MCH', 'WBC', 'HCT', 'PLT', 'MCHC', 'HGB', 'RBC', 'MCV'}


Scanning Anemic_individuals: 500it [00:01, 295.03it/s]

472_a.txt
{'HEMOGLOBIN', 'MCH', 'WBC', 'HCT', 'PLT', 'MCHC', 'RBC', 'MCV'}
470_a.txt
{'HEMOGLOBIN', 'MCH', 'WBC', 'HCT', 'PLT', 'MCHC', 'RBC', 'MCV'}
497_a.txt
{'HEMOGLOBIN', 'MCH', 'WBC', 'MPV', 'HCT', 'PLT', 'MCHC', 'RBC', 'MCV'}
496_a.txt
{'HEMOGLOBIN', 'MCH', 'WBC', 'HCT', 'PLT', 'MCHC', 'RBC', 'MCV'}
490_a.txt
{'HEMOGLOBIN', 'MCH', 'WBC', 'HCT', 'PLT', 'MCHC', 'RBC', 'MCV'}
494_a.txt
{'MCH', 'WBC', 'MPV', 'HCT', 'PLT', 'MCHC', 'HGB', 'RBC', 'MCV'}
486_a.txt
{'HEMOGLOBIN', 'MCH', 'WBC', 'HCT', 'PLT', 'MCHC', 'RBC', 'MCV'}
480_a.txt
{'HEMOGLOBIN', 'MCH', 'WBC', 'HCT', 'PLT', 'MCHC', 'RBC', 'MCV'}
487_a.txt
{'HEMOGLOBIN', 'MCH', 'WBC', 'HCT', 'PLT', 'MCHC', 'RBC', 'MCV'}
498_a.txt
{'HEMOGLOBIN', 'MCH', 'WBC', 'MPV', 'HCT', 'PLT', 'MCHC', 'RBC', 'MCV'}
493_a.txt
{'HEMOGLOBIN', 'MCH', 'WBC', 'MPV', 'HCT', 'PLT', 'MCHC', 'RBC', 'MCV'}
491_a.txt
{'HEMOGLOBIN', 'MCH', 'WBC', 'MPV', 'HCT', 'PLT', 'MCHC', 'RBC', 'MCV'}
492_a.txt
{'MCH', 'WBC', 'MPV', 'HCT', 'PLT', 'MCHC', 'HGB', 'RBC', 'MCV